# [Multi-Page Apps and URL Support](https://dash.plotly.com/urls)

マルチページアプリとURLサポート

Dash renders web applications as a "single-page app". When using `dcc.Link`, the application does not completely reload when navigating, making browsing very fast. Using Dash you can build multi-page apps using `dcc.Location` and `dcc.Link` components and callbacks.

Dash Pages uses these components and abstracts away the callback logic required for URL routing, making it easy to get up and running with a multi-page app. If you want to build a multi-page app without Pages, see the **Multi Page Apps without Pages** section below.

Dash は Web アプリケーションを "single-page app" としてレンダリングします。`dcc.Link` を使用すると、ナビゲーションの際にアプリケーションが完全にリロードされないので、ブラウジングが非常に高速になります。Dash を使用すると、`dcc.Location` と `dcc.Link` コンポーネントとコールバックを使用してマルチページアプリケーションを構築することができます。

Dash Pagesはこれらのコンポーネントを使用し、URLルーティングに必要なコールバックロジックを抽象化することで、マルチページアプリケーションを簡単に立ち上げることができます。もし、Pagesを使わずにマルチページアプリを構築したい場合は、以下の**Multi Page Apps without Pages**のセクションを参照してください。

## Dash Pages

>Dash Pages is new in Dash 2.5. Check your version with: `print(dash.__version__)`

Dash Pages is available from Dash version 2.5.0. It implements features to simplify creating a multi-page app, handling URL routing and offering an easy way to structure and define the pages in your app.

There are three basic steps for creating a multi-page app with Dash Pages:

- 1 Create individual `.py` files for each page in your app, and put them in a `/pages` directory.
- 2 In each of these page files:
  - Add a `dash.register_page(__name__)`, which tells Dash that this is a page in your app.
  - Define the page's content within a variable called `layout` or a function called `layout` that returns the content.
- 3 In your main app file, `app.py`:
  - When declaring your app, set `use_pages` to `True`: `app = Dash(__name__, use_pages=True)`
  - Add `dash.page_container` in your app layout where you want the page content to be displayed when a user visits one of the app's page paths.


>Dash 2.5では、Dash Pagesが新たに追加されました。バージョンを確認するには `print(dash.__version__)` で確認できます。

Dash Pages は、Dash バージョン 2.5.0 から利用できます。複数ページのアプリの作成、URL ルーティングの処理、アプリ内のページを簡単に構造化および定義する方法を提供する機能が実装されています。

Dash Pagesで複数ページのアプリを作るには、基本的に3つの手順があります。

- 1 アプリ内の各ページに個別の `.py` ファイルを作成し、`/pages` ディレクトリに配置します。
- 2 これらの各ページファイルでは
  - `dash.register_page(__name__)` を追加し、Dash にこれはあなたのアプリのページであることを伝えます。
  - ページのコンテンツは `layout` という変数か、コンテンツを返す `layout` という関数の中で定義してください。
- 3 アプリのメインファイルである `app.py` を作成します。
  - アプリの宣言時に `use_pages` を `True` に設定する： `app = Dash(__name__, use_pages=True)`
  - ユーザーがアプリのページパスのいずれかにアクセスしたときに、ページのコンテンツを表示したい場所に `dash.page_container` をアプリのレイアウトに追加します。

### Example: Simple Multi-Page App with Pages

Here is what a three page app structure looks like with Dash Pages:

### 例 ページによるシンプルなマルチページアプリ

Dash Pagesを使った3ページのアプリ構成はこんな感じです。

```bash
- app.py
- pages
   |-- analytics.py
   |-- home.py
   |-- archive.py
```

It has the main `app.py` file which is the entry point to our multi-page app (and in which we include `dash.page_container`) and three pages in our `pages` directory.

メインとなる `app.py` ファイルは、マルチページアプリのエントリポイントであり、 `dash.page_container` をインクルードしています。

`pages/analytics.py`:

In [ ]:
import dash
from dash import html, dcc, callback, Input, Output

dash.register_page(__name__)

layout = html.Div(children=[
    html.H1(children='This is our Analytics page'),
	html.Div([
        "Select a city: ",
        dcc.RadioItems(['New York City', 'Montreal','San Francisco'],
        'Montreal',
        id='analytics-input')
    ]),
	html.Br(),
    html.Div(id='analytics-output'),
])


@callback(
    Output(component_id='analytics-output', component_property='children'),
    Input(component_id='analytics-input', component_property='value')
)
def update_city_selected(input_value):
    return f'You selected: {input_value}'

`pages/home.py`:

In [ ]:
import dash
from dash import html, dcc

dash.register_page(__name__, path='/')

layout = html.Div(children=[
    html.H1(children='This is our Home page'),

    html.Div(children='''
        This is our Home page content.
    '''),

])

`pages/archive.py`:

In [ ]:
import dash
from dash import html, dcc

dash.register_page(__name__)

layout = html.Div(children=[
    html.H1(children='This is our Archive page'),

    html.Div(children='''
        This is our Archive page content.
    '''),

])

`app.py`:

In [ ]:
from dash import Dash, html, dcc
import dash

app = Dash(__name__, use_pages=True)

app.layout = html.Div([
	html.H1('Multi-page app with Dash Pages'),

    html.Div(
        [
            html.Div(
                dcc.Link(
                    f"{page['name']} - {page['path']}", href=page["relative_path"]
                )
            )
            for page in dash.page_registry.values()
        ]
    ),

	dash.page_container
])

if __name__ == '__main__':
	app.run_server(debug=True)

Notes:

**path**= — We call `dash.register_page` on each of the three pages in our app. For two of the pages, we don't set a `path` property. If you don't set the `path` property, it is autogenerated based on the module name. So `archives.py`'s layout is served when a user goes to `/archives`. Similarly, the layout for `analytics.py` is served when the user goes to `/analytics`. When we call `dash.register_page` for `home.py`, we do set the path property. For home.py we set the path property because we don't want the content to be displayed when the user goes to `/home`, but when the user goes to the homepage: `/`

**page_registry** — Pages that include a call to `dash.register_page` are added to the page registry for our app. This is an `OrderedDict` that we can extract information from about our app's pages. In our `app.py` we loop through all of our app's pages (in `dash.page_registry.values()`) and add links for each one. We can also select these links individually from the `dash.page_registry`. The page with `/` as the `path` is always in index `0` in the dict. Other pages are in alphabetical order.

**page_container** —`app.py` has a `dash.page_container`. This is where page content is displayed when a user navigates to that page's path.

**path**= - アプリ内の3つのページそれぞれで、 `dash.register_page` を呼び出します。そのうち2つのページでは、`path` プロパティを設定しません。もし、`path` プロパティを設定しなければ、モジュール名に基づいて自動生成されます。そのため、ユーザーが `/archives` にアクセスすると、 `archives.py` のレイアウトが提供されます。同様に、`analytics.py`のレイアウトは、ユーザーが`/analytics`にアクセスしたときに提供されます。`home.py`に対して `dash.register_page` を呼び出すときには、pathプロパティを設定します。home.pyでは、ユーザーが`/home`にアクセスしたときにコンテンツを表示するのではなく、ユーザーがホームページにアクセスしたときにコンテンツを表示するために、pathプロパティを設定します。`/`

**page_registry** - `dash.register_page` への呼び出しを含むページが、アプリのページレジストリに追加されます。これは `OrderedDict` で、アプリのページに関する情報を抽出することができます。`app.py` では、アプリの全てのページ（`dash.page_registry.values()`）をループして、それぞれのページにリンクを追加しています。また、これらのリンクは `dash.page_registry` から個別に選択することができます。パスが `/` であるページは、常に dict のインデックス `0` に位置します。その他のページはアルファベット順です。

**page_container** -`app.py` は `dash.page_container` を持ちます。これは、ユーザーがそのページのパスにナビゲートしたときに、ページのコンテンツが表示される場所です。

### Layout

In the above example, we've defined the layout in each page using a variable called `layout`. For example, in `home.py` above:

### レイアウト

上記の例では、各ページのレイアウトを `layout` という変数で定義しています。例えば、上記の`home.py`の場合。

In [ ]:
layout = html.Div(children=[
    html.H1(children='This is our Home page'),

    html.Div(children='''
        This is our Home page content.
    '''),

])

You can also use a function called `layout` that returns your page content:

また、ページの内容を返す `layout` という関数も使用できます。

In [ ]:
def layout():
    return html.Div(children=[
    html.H1(children='This is our Home page'),

    html.Div(children='''
        This is our Home page content.
    '''),

])

>Page layouts must be defined with a variable or function called `layout`. When creating an app with Pages, only use `app.layout` in your main `app.py` file.

>ページのレイアウトは、 `layout` という変数や関数で定義する必要があります。Pages を使ってアプリを作成する場合、メインの `app.py` ファイルでは `app.layout` のみを使用します。

### dash.register_page

Calling `dash.register_page` within a file is how Dash knows to include the file as a page in your multi-page app.

As we've seen, it can be called with just the module name:

### dash.register_page

ファイル内で `dash.register_page` を呼び出すことで、Dash はそのファイルをマルチページアプリの 1 ページとしてインクルードするかどうかを判断します。

これまで見てきたように、これはモジュール名だけで呼び出すことができます。

In [ ]:
dash.register_page(__name__)

In this case, Dash generates the `path` the page is for, its `title`, and the link `name` based on the module name.

この場合、Dash はモジュールの名前に基づいて、ページの `path` と `title` 、そしてリンクの `name` を生成します。

>The `title` is the **HTML `<title>`**. The `name` is the key for this page in the Dash Registry and can be used when creating links for pages. The path is the URL pathname of the page.

>`title` は **HTML の `<title>`** です。`name` はDashレジストリにおけるこのページのキーであり、ページへのリンクを作成する際に使用することができます。path はページのURLパス名です。

So, if we have a file called `analytics.py`, the page's path is `/analytics`, the title is `Analytics`, and the link name is `Analytics`.

We can also specify these if we don't want them to be autogenerated based on the module name, as we did in the example above with our home page.

Setting a path, title, and link name:

つまり、 `analytics.py` というファイルがあれば、ページのパスは `/analytics` で、タイトルは `Analytics` で、リンク名は `Analytics` となります。

上記のホームページの例のように、モジュール名から自動生成させたくない場合は、これらを指定することも可能です。

パス、タイトル、リンク名の設定:

`pages/analytics.py`

In [ ]:
dash.register_page(
    __name__,
    path='/analytics-dashboard',
    title='Our Analytics Dashboard',
    name='Our Analytics Dashboard'
)

See the **Reference for dash.register_page** section below for a detailed list of properties.

プロパティの詳細については、以下の「**dash.register_page**のリファレンス」セクションを参照してください。

### Dash Page Registry

Any pages that call `dash.register_page` are added to a page registry for your app.

The page registry is an `OrderedDict` called `dash.page_registry`. Each registry entry has information for a page, including property values set when `dash.register_page` was called, and values inferred by Dash. As with any dict, you can access and use its data in your code.

Here we access the `path` of our `analytics` and use it in a dcc.Link in `app.py`:

### Dash Page Registry

`dash.register_page` を呼び出したすべてのページは、アプリのページレジストリに追加されます。

ページレジストリは `dash.page_registry` という名前の `OrderedDict` です。各レジストリ項目には、 `dash.register_page` が呼ばれたときに設定されたプロパティ値や、Dash が推測した値など、ページに関する情報が含まれています。他のディクショナリーと同様に、コード内でそのデータにアクセスし、使用することができます。

ここでは、 `analytics` の `path` にアクセスし、それを `app.py` の dcc.Link で使用しています。

In [ ]:
html.Div(dcc.Link('Dashboard', href=dash.page_registry['pages.analytics']['path']))

To access `dash.page_registry` from within a file in the `pages` directory, you'll need to use it within a function.

`pages` ディレクトリにあるファイルから `dash.page_registry` にアクセスするには、関数内でこの関数を使用する必要があります。

Here, we have two files within the `pages` directory: `side_bar.py` and `topic_1.py`. The `topic_1` page imports a sidebar from `side_bar.py`. Note how the function within `side_bar.py` accesses `dash.page_registry`. If this wasn't within a function, the app wouldn't work because the `dash.page_registry` wouldn't be ready when the page loads.

ここでは、`pages` ディレクトリに `side_bar.py` と `topic_1.py` という2つのファイルを置いています。`topic_1` ページは `side_bar.py` からサイドバーをインポートしています。`side_bar.py`内の関数が、どのように `dash.page_registry` にアクセスしているかに注意してください。もしこれが関数内でなければ、ページがロードされたときに `dash.page_registry` が準備されていないため、アプリは動作しないでしょう。

`side_bar.py`

In [ ]:
import dash
from dash import html
import dash_bootstrap_components as dbc


def sidebar():
    return html.Div(
        dbc.Nav(
            [
                dbc.NavLink(
                    [
                        html.Div(page["name"], className="ms-2"),
                    ],
                    href=page["path"],
                    active="exact",
                )
                for page in dash.page_registry.values()
                if page["path"].startswith("/topic")
            ],
            vertical=True,
            pills=True,
            className="bg-light",
        )
    )

`topic_1.py`

In [ ]:
from dash import html

import dash
import dash_bootstrap_components as dbc

from .side_bar import sidebar

dash.register_page(
    __name__,
    name="Topics",
    top_nav=True,
)


def layout():
    return dbc.Row(
        [dbc.Col(sidebar(), width=2), dbc.Col(html.Div("Topics Home Page"), width=10)]
    )

What the Dash Page Registry looks like for our initial example, Simple Multi-page App with Pages

Dash Page Registryは、最初の例であるSimple Multi-page App with Pagesの場合、どのように見えるでしょうか。

In [ ]:
OrderedDict([('pages.home',
              {'description': '',
               'image': None,
               'layout': Div([H1('This is our Home page'), Div('\n        This is our Home page content.\n    ')]),
               'module': 'pages.home',
               'name': 'Home',
               'order': 0,
               'path': '/',
               'path_template': None,
               'redirect_from': None,
               'relative_path': '/',
               'supplied_image': None,
               'supplied_layout': None,
               'supplied_name': None,
               'supplied_order': None,
               'supplied_path': '/',
               'supplied_title': None,
               'title': 'Home'}),
             ('pages.analytics',
              {'description': '',
               'image': None,
               'layout': Div([H1('This is our Analytics page'), Div(['Select a city: ', RadioItems(options=['New York City', 'Montreal', 'San Francisco'], value='Montreal', id='analytics-input')]), Br(None), Div(id='analytics-output')]),
               'module': 'pages.analytics',
               'name': 'Analytics',
               'order': None,
               'path': '/analytics',
               'path_template': None,
               'redirect_from': None,
               'relative_path': '/analytics',
               'supplied_image': None,
               'supplied_layout': None,
               'supplied_name': None,
               'supplied_order': None,
               'supplied_path': None,
               'supplied_title': None,
               'title': 'Analytics'}),
             ('pages.archive',
              {'description': '',
               'image': None,
               'layout': Div([H1('This is our Archive page'), Div('\n        This is our Archive page content.\n    ')]),
               'module': 'pages.archive',
               'name': 'Archive',
               'order': None,
               'path': '/archive',
               'path_template': None,
               'redirect_from': None,
               'relative_path': '/archive',
               'supplied_image': None,
               'supplied_layout': None,
               'supplied_name': None,
               'supplied_order': None,
               'supplied_path': None,
               'supplied_title': None,
               'title': 'Archive'})])

#### Dash Page Registry Order

By default, a page with a path defined as '/' is added to the registry at index `0`. Other pages are then added in alphabetical order based on file name.

You can also specify the order of pages in `dash.page_registry` by setting the `order` property on each page:

#### Dash Page Registry Order

デフォルトでは、パスが '/' で定義されているページが、インデックス `0` としてレジストリに追加されます。その後、他のページがファイル名に基づいたアルファベット順に追加されます。

また、各ページに `order` プロパティを設定することで、 `dash.page_registry` に登録されるページの順番を指定することができます。

`pages/analytics.py`:

In [ ]:
dash.register_page(__name__, order=3)

If you set the `order` property on one or more pages, pages are added to the registry:

- In the order they are specified with the `order` property.
- In alphabetical order after that (for pages without the order property set.

Setting the order can be useful when you want to be able to loop through the links when creating a sidebar or header dynamically.

1つまたは複数のページに `order` プロパティを設定すると、レジストリにページが追加されます。

- `order` プロパティで指定された順番に。
- `order` プロパティで指定された順番で、それ以降はアルファベット順（orderプロパティが設定されていないページの場合。

orderを設定すると、サイドバーやヘッダーを動的に作成する際に、リンクをループさせることができるようになり便利です。

### Default and Custom 404

If a user goes to a path that hasn't been declared in one of your app's pages, Pages shows a default **'404 - Page not found message'** to the user.

This page can be customized. Place a file called `not_found_404.py` in your app's `pages` directory, add `dash.register_page(__name__)` to the file, and define the content for the custom 404 within a `layout` variable or function:

### Default and Custom 404

ユーザーがアプリのページで宣言されていないパスに移動した場合、Pages はデフォルトの **'404 - Page not found message'** をユーザーに表示します。

このページはカスタマイズすることができます。アプリの `pages` ディレクトリに `not_found_404.py` というファイルを置き、そのファイルに `dash.register_page(__name__)` を追加して、カスタム 404 のコンテンツを `layout` 変数または関数内で定義します。

In [ ]:
from dash import html
import dash

dash.register_page(__name__)

layout = html.H1("This is our custom 404 content")

### Variable Paths

You can capture dynamic variables in the path by using the `path_template` parameter. Specify dynamic parts of your URL by placing it within `<variable_name>`. `variable_name` will be the named keyword argument passed into your layout function. Values that the layout function receives from the URL are always of type `str`.

### Variable Paths

`path_template` パラメータを使用することで、動的な変数をパスに取り込むことができます。URLの動的な部分を `<variable_name>` 内に記述して指定します。`variable_name` はレイアウト関数に渡される名前付きキーワード引数になります。レイアウト関数がURLから受け取る値は、常に `str` 型です。

#### Example - Single Variable Path

In [ ]:
import dash
from dash import html

dash.register_page(__name__, path_template="/report/<report_id>")


def layout(report_id=None):
    return html.Div(
        f"The user requested report ID: {report_id}."
    )

#### Example - Two Path Variables and Update Title & Description

The path variables can also be used to update the page's `title` (what you see in the browser tab) and the page's meta `description` (information used by search engines when indexing and displaying search results and also displayed in social media when sharing links; otherwise not visible). More information on these parameters can be found in the **Reference for dash.register_page** section below.

#### 例 - 2つのパス変数とタイトルと説明文の更新

パス変数は、ページの `title` (ブラウザのタブに表示されるもの) とページのメタ `description` (検索エンジンが検索結果をインデックス化および表示する際に使用する情報、およびリンクを共有する際にソーシャルメディアに表示される情報、それ以外は表示されない) を更新するためにも使用することができます。これらのパラメータに関する詳細は、以下の **dash.register_page** のリファレンスを参照してください。

In [ ]:
import dash

def title(asset_id=None, dept_id=None):
    return f"Asset Analysis: {asset_id} {dept_id}"


def description(asset_id=None, dept_id=None):
    return f"This is the AVN Industries Asset Analysis: {asset_id} in {dept_id}"


dash.register_page(
    __name__,
    path_template="/asset/<asset_id>/department/hello-<dept_id>",
    title=title,
    description=description,
)


def layout(asset_id=None, dept_id=None, **other_unknown_query_strings):
    return dash.html.Div(
        f"variables from pathname:  asset_id: {asset_id} dept_id: {dept_id}"
    )

### Query Strings

Query string parameters in a URL can be captured by Pages.

#### Example - Single Query String Parameter

In this example, when the user goes to `/archive?report_id=9`, the value `9` is captured by the layout function and displayed on the page. Values that the layout function receives from the URL are always of type `str`.

### Query Strings

URLに含まれるクエリー文字列のパラメーターをPagesで取得することができます。

#### 例 - 単一のクエリ文字列パラメータ

この例では、ユーザーが `/archive?report_id=9` にアクセスすると、値 `9` がレイアウト関数に取り込まれ、ページ上に表示されます。レイアウト機能がURLから受け取る値は、常に `str` 型です。

In [ ]:
import dash
from dash import html

dash.register_page(__name__)

def layout(report_id=None, **other_unknown_query_strings):
    return html.Div(
	children=[
	    html.H1(children='This is our Archive page'),

	    html.Div(children=f'''
	        This is report: {report_id}.
	    '''),

	])

#### Example - Two Query String Parameters

In this example, when the user goes to `/archive?report_id=9&department_id=55`, the values `9` and `55` are captured by the layout function and displayed on the page.

#### 例 - 2つのクエリ文字列パラメータ

この例では、ユーザーが `/archive?report_id=9&department_id=55` にアクセスすると、値 `9` と `55` がレイアウト関数に取り込まれ、ページに表示されることになります。

In [ ]:
import dash
from dash import html

dash.register_page(__name__)

def layout(report_id=None, department_id=None, **other_unknown_query_strings):
    return html.Div(
	children=[
	    html.H1(children='This is our Archive page'),

	    html.Div(children=f'''
	        This is report: {report_id}.\n
			This is department: {department_id}.
	    '''),

	])

### Redirects

If you change a page's path, it's best practice to define a redirect so users that go to old links don't get a '404 – Page not found'. You can set additional paths to direct to a page using the `redirects` parameter. This takes a `list` of all paths that redirect to this page.

Here we have a page called `archive`. It is displayed when a user goes to `/archive`, `/archive-2021`, or `/archive-2020`

### Redirects

ページのパスを変更した場合、古いリンクにアクセスしたユーザーが '404 - Page not found' を受け取らないように、リダイレクトを定義することがベストプラクティスです。`redirects` パラメータを使用すると、ページに誘導する追加のパスを設定することができます。このパラメータには、このページにリダイレクトするすべてのパスの `list` を指定します。

ここでは、 `archive` という名前のページがあります。これは、ユーザが `/archive`, `/archive-2021`, または `/archive-2020` にアクセスしたときに表示されるページです。

`archive.py`

In [ ]:
import dash
from dash import html, dcc

dash.register_page(
    __name__,
    path="/archive",
    redirect_from=["/archive-2021", "/archive-2020"]
)


layout = html.Div(children=[
    html.H1(children='This is our Archive page'),

    html.Div(children='''
        This is our Archive page content.
    '''),

])

### Meta Tags

>Not sure what meta tags are? [Check out this tutorial on meta tags and why you might want to use them](https://developer.mozilla.org/en-US/docs/Learn/HTML/Introduction_to_HTML/The_head_metadata_in_HTML).

Each page you add to your app has page meta tags stored for it: a `title`, `image`, and `description`.

The `title` is used as the page title in the browser, but together with the `image` and `description`, it is also often used by social media sites and chat clients to create a card to display when someone shares a link to a page.

You can set the values for these properties with `title=`, `description=`, `image=`:

### Meta Tags

>metaタグが何であるかわからない？[metaタグのチュートリアルと、なぜmetaタグを使いたがるのかをご覧ください。](https://developer.mozilla.org/en-US/docs/Learn/HTML/Introduction_to_HTML/The_head_metadata_in_HTML)

アプリに追加した各ページには、ページのmetaタグが保存されています: `title`、`image`、`description`です。

`title` はブラウザ上でページのタイトルとして使用されますが、 `image` や `description` と共に、ソーシャルメディアサイトやチャットクライアントで、誰かがページへのリンクを共有したときに表示するカードを作成するためにもよく使用されます。

これらのプロパティに値を設定するには、 `title=`、`description=`、`image=` を使用します。

In [ ]:
dash.register_page(__name__, title='Custom Page Title', description='Custom Page Description', image='logo.png')

Image types of `apng`, `avif`, `gif`, `jpeg`, `jpg`, `png`, `svg`, and `webp` are supported.

The `image` value must be the name of a file inside the `assets` folder. To set the `image` to a file that is not in the `assets` folder, such as an image hosted externally on a CDN, change `image=` to `image_url=` and provide the URL.

If you don't specify `title`, it is derived from the module name. If you don't specify a `description`, it defaults to `None`. Lastly, if you don't specify `image`, Pages checks for an image that meets one of these criteria (in order) and uses the first one it finds:

- A page-specific image at `assets/<module>.<extension>`
- A generic app image at `assets/app.<extension>`
- A logo at `assets/logo.<extension>`

For example, placing a file `analytics.png` in the `assets` folder sets this file as the `image` for `pages/analytics.py`, because the first criterion is met.

A more complete example for setting meta tags with Pages might look like:

画像の種類としては、 `apng`, `avif`, `gif`, `jpeg`, `jpg`, `png`, `svg`, `webp` がサポートされています。

`image` の値には、 `assets` フォルダ内のファイル名を指定する必要があります。CDN などの外部でホストされている画像のように、 `assets` フォルダ内にないファイルを `image` に設定するには、 `image=` を `image_url=` に変更して、その URL を指定してください。

`title` を指定しない場合は、モジュール名から派生したものとなります。`description` を指定しない場合、デフォルトは `None` です。最後に、 `image` を指定しなかった場合、Pages はこれらの条件を満たす画像を (順番に) チェックし、最初に見つかった画像を使用します。

- `assets/<module>.<extension>` にあるページ固有の画像
- 汎用的なアプリのイメージは `assets/app.<extension>` にあります。
- `assets/logo.<extension>`にあるロゴ。

例えば、`assets`フォルダに `analytics.png` というファイルを置くと、最初の条件が満たされるため、このファイルが `pages/analytics.py` の `image` として設定されます。

Pagesでmetaタグを設定する、より完全な例は、次のようになります。

In [ ]:
import dash
from dash import html, dcc


dash.register_page(
	__name__,
    title='Explore the archive',
    image='archive_image_2022.png',
    description='The archive page shows previously generated reports.'
)

layout = html.Div(children=[
    html.H1(children='This is our Archive page'),

    html.Div(children='''
        This is our Archive page content.
    '''),

])

The `title` and `description` properties can also be set as functions. If provided as functions, Pages calls these functions on page load and uses the values that they return.

`title` と `description` プロパティは、関数として設定することもできます。関数として提供された場合、Pages はページロード時にこれらの関数を呼び出し、それらが返す値を使用します。

### Additional Keywords with Dash Page Registry

You can use additional custom key-value pairs when calling `dash.register_page` to add those to the Dash Page Registry.

For example, if you want to add information to a page about where its links appear on the home page, you could add a "location" keyword with a value of "sidebar".

### Dash Page Registry による追加キーワード

`dash.register_page` を呼び出すときに、追加のカスタムキーと値のペアを使用して、Dash Page Registry にそれらを追加することができます。

例えば、ホームページのどこにリンクが表示されるかという情報をページに追加したい場合、"location" キーワードを "sidebar" という値で追加することができます。

In [ ]:
dash.register_page(__name__, location = "sidebar")

In your `app.py` page, in the sidebar, you can then loop through the pages that have that location set:

サイドバーの `app.py` ページで、そのロケーションが設定されているページをループさせることができます。

In [ ]:
html.Div(
    [
        html.Div(
            dcc.Link(
                f"{page['name']}", href=page["path"]
            )
        )
        for page in dash.page_registry.values() if page["location"] == "sidebar"
    ]
)

When you add new pages with `dash.register_page(__name__, location = "sidebar")`, they'll automatically be included in the sidebar.

`dash.register_page(__name__, location = "sidebar")` で新しいページを追加すると、それらは自動的にサイドバーに含まれるようになります。

### Nested Pages

Dash Pages also recursively searches directories in the `pages` directory for additional app pages. For example, if we add a `reports` (this name is arbitrary!) directory within pages, put two pages, `summary_2020.py` and `summary_2021.py`, in that directory, and call `dash.register_page(__name__)`, they will be included in our app.

### ネストされたページ

また、Dash Pages は `pages` ディレクトリの中にあるディレクトリを再帰的に検索して、アプリのページを追加していきます。例えば、pages ディレクトリ内に `reports` (この名前は任意です!) ディレクトリを追加し、そのディレクトリに `summary_2020.py` と `summary_2021.py` という二つのページを置いて、 `dash.register_page(__name__)` と呼べば、それらが我々のアプリに含まれることになります。

```bash
- app.py
- pages
    - reports
        |-- summary_2020.py
        |-- summary_2021.py
   |-- analytics.py
   |-- home.py
   |-- archive.py
```

`pages/reports/summary_2020.py`:

In [ ]:
import dash
from dash import html, dcc

dash.register_page(
	__name__,
)


layout = html.Div(children=[
    html.H1(children='2020 Summary'),

    html.Div(children='''
        This is our page's content.
    '''),

])

As we haven't set the `path` property, Pages will display this page when the user visits the app at the URL path `/reports/summary-2020`.

`path`プロパティを設定していないので、ユーザーがURLパス `/reports/summary-2020` でアプリにアクセスすると、Pagesはこのページを表示します。

### Changing the Default Pages Directory

By default, Pages checks for a directory called `pages` for your app files. You can changes this when declaring your Dash app:

### デフォルトのページディレクトリを変更する

デフォルトでは、Pages は `pages` というディレクトリをチェックし、そこにアプリのファイルを置きます。これはDashアプリを宣言する際に変更することができます。

In [ ]:
app = dash.Dash(__name__, use_pages=True, pages_folder="my_apps")

### Multiple Pages in One File

So far, we've built a multi-page app where we've declared each page in a separate `.py` file in our `pages` directory. It's also possible to declare multiple pages within `app.py`.

To do this, we register the page within `app.py` and pass the layout directly to `dash.register_page`. In this example, we define two pages within our app.py file: a `home` page, and an `analytics` page. For `module`, the first argument, we give each of our pages a unique name (as these names are used as keys in the Dash Page Registry).

### 1つのファイルに複数のページがある

これまで、私たちは複数ページのアプリを作成し、各ページを `pages` ディレクトリにある別の `.py` ファイルで宣言してきました。また、`app.py`の中で複数のページを宣言することも可能です。

そのためには、`app.py`の中でページを登録し、レイアウトを直接 `dash.register_page` に渡します。この例では、`home`ページと`analytics`ページの2つのページをapp.pyファイル内で定義します。最初の引数である `module` には、それぞれのページにユニークな名前を付けます（これらの名前は、Dash Page Registry のキーとして使用されます）。

In [ ]:
from dash import Dash, html, dcc, callback
import dash

app = Dash(__name__, use_pages=True)

dash.register_page("home",  path='/', layout=html.Div('Home Page'))
dash.register_page("analytics", layout=html.Div('Analytics'))

app.layout = html.Div([
    html.Div(
        [
            html.Div(
                dcc.Link(
                    f"{page['name']} - {page['path']}", href=page["relative_path"]
                )
            )
            for page in dash.page_registry.values()
        ]
    ),
    dash.page_container,
])


if __name__ == '__main__':
    app.run_server(debug=True)

### Circular Imports

When using Pages, the file that declares `Dash(__name__, use_pages=True)` recursively imports all files within the `pages` folder. If any of those pages import a function from the Dash file (usually `app.py`), then you will get a circular import error.

For example, this will cause a circular import error:

### サーキュラー・インポート

Pages を使用する場合、 `Dash(__name__, use_pages=True)` と宣言したファイルは、 `pages` フォルダ内のすべてのファイルを再帰的にインポートします。もし、それらのページのどれかが Dash ファイル(通常は `app.py`) の関数をインポートしていた場合、循環的なインポートエラーが発生します。

例えば、次のようにすると、循環インポートエラーが発生します。

`app.py`

In [ ]:
import dash
from dash import html

app = dash.Dash(
    __name__,
    use_pages=True
)

app.layout = html.Div(
    dash.page_container
)


if __name__ == "__main__":
    app.run_server(debug=True)

`analytics.py`

In [ ]:
from dash import Input, Output, html, dcc
from app import app

layout = html.Div([dcc.Input(id='input'), html.Div(id='output')])

@app.callback(Output('output', 'children'), Input('input', 'value'))
def update(value):
    return value

Running `python app.py` displays the error `KeyError: 'pages.analytics'`

- If you are trying to use a callback within one of your files in the `pages` directory, you can instead use `dash.callback`:

python app.py` を実行すると、エラー `KeyError: 'pages.analytics'` が表示されます。

- もし、`pages` ディレクトリにあるファイルの中でコールバックを使用しようとしているのであれば、代わりに `dash.callback` を使用することができます。

In [ ]:
import dash
from dash import Input, Output, html, dcc

dash.register_page(
    __name__
)
layout = html.Div([dcc.Input(id='input'), html.Div(id='output')])

@dash.callback(Output('output', 'children'), Input('input', 'value'))
def update(value):
    return value

- In cases where you need access to the `app` in one of your page layouts, you can access it with `dash.get_app`.
- If you want to use `app.get_asset_url` in one of your pages, use `dash.get_asset_url`.

- ページレイアウトの中で `app` にアクセスする必要がある場合、 `dash.get_app` を使ってアクセスすることができます。
- もし、ページの中で `app.get_asset_url` を使いたい場合は、 `dash.get_asset_url` を使ってください。

### Reference for dash.register_page

Assigns the variables to `dash.page_registry` as an `OrderedDict` (ordered by `order`).

`dash.page_registry` is used by `pages_plugin` to set up the layouts as a multi-page Dash app. This includes the URL routing callbacks (using `dcc.Location`) and the HTML templates to include title, meta description, and the meta description image.

`dash.page_registry` can also be used by Dash developers to create the page navigation links or by template authors.


- `module`: The module path where this page's `layout` is defined. Often `__name__`.

- `path`: URL Path, e.g. `/` or `/home-page`. If not supplied, will be inferred from the `path_template` or `module`, e.g. based on path_template: `/asset/<asset_id` to `/asset/none` e.g. based on module: `pages.weekly_analytics` to `/weekly-analytics`

 - `relative_path`: The path with `requests_pathname_prefix` prefixed before it. Use this path when specifying local URL paths that will work in environments regardless of what `requests_pathname_prefix` is. In some deployment environments, like Dash Enterprise, `requests_pathname_prefix` is set to the application name, e.g. `my-dash-app`. When working locally, `requests_pathname_prefix` might be unset and so a relative URL like `/page-2` can just be `/page-2`. However, when the app is deployed to a URL like `/my-dash-app`, then `relative_path` will be `/my-dash-app/page-2`.

- `path_template`: Add variables to a URL by marking sections with <variable_name>. The layout function then receives the <variable_name> as a keyword argument. e.g. path_template= "/asset/<asset_id>" then if pathname in browser is "/assets/a100" then layout will receive **{"asset_id":"a100"}

- `name`: The name of the link. If not supplied, will be inferred from `module`, e.g. `pages.weekly_analytics` to `Weekly analytics`

- `order`: The order of the pages in `page_registry`. If not supplied, then the filename is used and the page with path `/` has order `0`

- `title`: (string or function) The name of the page . That is, what appears in the browser title. If not supplied, will use the supplied `name` or will be inferred by module, e.g. `pages.weekly_analytics` to `Weekly analytics`

- `description`: (string or function) The . If not supplied, then nothing is supplied.

- `image`: The meta description image used by social media platforms. If not supplied, then it looks for the following images in `assets/`:

  - A page specific image: `assets/<module>.<extension>` is used, e.g. `assets/weekly_analytics.png`

  - A generic app image at `assets/app.<extension>`

  - A logo at `assets/logo.<extension>` When inferring the image file, it will look for the following extensions: APNG, AVIF, GIF, JPEG, JPG, PNG, SVG, WebP.

- `image_url`: Overrides the image property and sets the `<image>` meta tag to the provided image URL.

- `redirect_from`: A list of paths that should redirect to this page. For example: `redirect_from=['/v2', '/v3']`

- `layout`: The layout function or component for this page. If not supplied, then looks for `layout` from within the supplied `module`.

- `**kwargs`: Arbitrary keyword arguments that can be stored.

### dash.register_pageのリファレンス

`dash.page_registry` に変数を `OrderedDict` (`order` で並べたもの) として代入します。

`dash.page_registry` は `pages_plugin` によって、複数ページのDashアプリとしてレイアウトをセットアップするために使用されます。これには、URLルーティングコールバック（`dcc.Location`を使用）と、タイトル、メタディスクリプション、メタディスクリプションイメージを含むHTMLテンプレートが含まれます。

また、`dash.page_registry`はDash開発者がページナビゲーションリンクを作成したり、テンプレート作成者が使用したりすることができます。


- `module`: このページの `layout` が定義されているモジュールのパスです。多くの場合、 `__name__` です。

- `path`: URL パス。例えば `/` や `/home-page` など。もし提供されていなければ、 `path_template` や `module` から推測さ れます。例えば path_template に基づいている場合は、 `/asset/<asset_id` から `/asset/none` へ、例えば module に基づいている場合は、 `/asset/none` へ。例: `pages.weekly_analytics` を `/weekly-analytics` に変更する。

- `relative_path`: `requests_pathname_prefix` を前に付けたパス。このパスは、 `requests_pathname_prefix` が何であるかに関係なく、環境で動作するローカル URL パスを指定するときに使用します。Dash Enterprise のようないくつかのデプロイ環境では、 `requests_pathname_prefix` は `my-dash-app` などのようにアプリケーション名に設定されます。ローカルで作業しているときは、`requests_pathname_prefix` が設定されていないため、`/page-2` のような相対 URL は `/page-2` だけでよい場合があります。しかし、アプリを `/my-dash-app` のような URL にデプロイした場合、 `relative_path` は `/my-dash-app/page-2` となります。

- `path_template` を使用します。<variable_name> でセクションをマークすることにより、URLに変数を追加します。例えば、path_template= "/asset/<asset_id>" とすると、ブラウザのパス名が "/assets/a100" であれば、レイアウトは **{"asset_id": "a100"} を受け取ることになります。

- `name`: リンクの名前。省略した場合は、 `module` から推測されます。例えば、 `pages.weekly_analytics` は `Weekly analytics` になります。

- `order`: `page_registry` に登録されているページの順番。指定しない場合は、ファイル名が使用され、パスが `/` のページが `0` の順番になります。

- `title`: (文字列または関数) ページの名前。 つまり、ブラウザのタイトルに表示されるものです。もし提供されていなければ、提供された `name` を使用するか、モジュールによって推測されます。例えば、 `pages.weekly_analytics` は `Weekly analytics` に変換されます。

- `description`: (文字列または関数) . 与えられなければ、何も提供されません。

- `image`: ソーシャルメディアプラットフォームで使用されるメタディスクリプションの画像です。もし提供されていない場合は、`assets/` にある以下の画像を探します。

  - ページ固有の画像です。例えば、`assets/<module>.<extension>`が使用されます（例： `assets/weekly_analytics.png` ）。

  - `assets/app.<extension>` にある汎用アプリ画像

  - `assets/logo.<extension>` にあるロゴ イメージファイルを推測するとき、以下の拡張子を探します。APNG、AVIF、GIF、JPEG、PNG、SVG、WebP。

- `image_url` を指定します。image プロパティをオーバーライドし、 `<image>` メタタグに指定された画像の URL を設定します。

- `redirect_from`: このページにリダイレクトするパスのリスト。例: `redirect_from=['/v2', '/v3']`。

- `layout`: このページのレイアウト機能またはコンポーネント。もし提供されていない場合は、提供された `module` の中から `layout` を探します。

- `**kwargs`: 保存可能な任意のキーワード引数。

`page_registry` stores the original property that was passed in under `supplied_<property>` and the coerced property under `<property>`. For example, if this was called:

`page_registry` には、渡された元のプロパティが `supplied_<property>` の下に、強制されたプロパティが `<property>` の下に格納されます。例えば、これが呼び出された場合。

In [ ]:
register_page(
    'pages.historical_outlook',
    name='Our historical view',
    custom_key='custom value'
)

Then this will appear in `page_registry`:

そうすると、これは `page_registry` に表示されます。

In [ ]:
OrderedDict([
    (
        'pages.historical_outlook',
        dict(
            module='pages.historical_outlook',

            supplied_path=None,
            path='/historical-outlook',

            supplied_name='Our historical view',
            name='Our historical view',

            supplied_title=None,
            title='Our historical view'

            supplied_layout=None,
            layout=<function pages.historical_outlook.layout>,

            custom_key='custom value'
        )
    ),
])

## Multi-Page Apps without Pages

>Dash Pages (available in Dash 2.5 and later) is the easiest way to build a multi-page app in Dash. If you are using an earlier version of Dash 2.x, you can build a multi-page app using the following guide.
>
>Dash Pages uses a `dcc.Location` callback under-the-hood as described in the method below. Dash Pages also automatically:
>
>- Sets configurable `title`, `description`, and image meta tags using [interpolate_index](https://dash.plotly.com/external-resources) and clientside callbacks under-the-hood
>
>- Sets configurable redirects using `flask.redirect` under-the-hood
>
>- Sets configurable 404 content
>
>- Sets `validate_layout` under-the-hood to avoid callback exceptions See the [community announcement for the original discussion of this feature](https://community.plotly.com/t/introducing-dash-pages-a-dash-2-x-feature-preview/57775?_gl=1*7ftafz*_ga*MTU4NzAyNjYyMC4xNjY4NjM2MzMw*_ga_6G7EE0JNSC*MTY2OTU4NzA5My41Ny4xLjE2Njk1ODcxMTQuMC4wLjA.).

## ページを持たないマルチページアプリ

>Dash Pages（Dash 2.5 以降で利用可能）は、Dash で複数ページのアプリを構築する最も簡単な方法です。Dash 2.x の以前のバージョンを使用している場合は、次のガイドを使用して複数ページのアプリを構築できます。
>
>Dash Pagesは、以下のメソッドにあるように、アンダーザフッドで `dcc.Location` コールバックを使用します。また、Dash Pagesは自動的に:
>
>- [interpolate_index](https://dash.plotly.com/external-resources) とクライアントサイドのコールバックを使用して、設定可能な `title`, `description`, および画像のメタタグを設定します。
>
>- アンダーザフードの `flask.redirect` を使って、設定可能なリダイレクトを設定します。
>
>- コールバックの例外を避けるために、アンダーザフードで `validate_layout` を設定します。 この機能に関するオリジナルの議論は、[コミュニティアナウンスメント](https://community.plotly.com/t/introducing-dash-pages-a-dash-2-x-feature-preview/57775?_gl=1*7ftafz*_ga*MTU4NzAyNjYyMC4xNjY4NjM2MzMw*_ga_6G7EE0JNSC*MTY2OTU4NzA5My41Ny4xLjE2Njk1ODcxMTQuMC4wLjA.) を参照してください。

The components [dcc.Location](https://dash.plotly.com/dash-core-components/location) and [dcc.Link](https://dash.plotly.com/dash-core-components/link) aid page navigation: `dcc.Location` represents the web browser address bar. You can access the current pathname in the user's browser with `dcc.Location`'s `pathname` property. `dcc.Link` updates the `pathname` in the browser.

In the following examples, we demonstrate using these components to build multi-page apps, without using Dash Pages.

[dcc.Location](https://dash.plotly.com/dash-core-components/location)と[dcc.Link](https://dash.plotly.com/dash-core-components/link)というコンポーネントはページのナビゲーションを補助します： `dcc.Location` はウェブブラウザのアドレスバーを表します。`dcc.Location` はウェブブラウザのアドレスバーを表し、 `dcc.Location` の `pathname` プロパティでユーザのブラウザにある現在のパス名にアクセスすることができます。`dcc.Link` はブラウザの `pathname` を更新します。

以下の例では、これらのコンポーネントを使用して、Dash Pagesを使用しないマルチページアプリケーションを構築するデモを行います。

### Simple Example

In [ ]:
from dash import Dash, dcc, html, callback, Input, Output

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    # represents the browser address bar and doesn't render anything
    dcc.Location(id='url', refresh=False),

    dcc.Link('Navigate to "/"', href='/'),
    html.Br(),
    dcc.Link('Navigate to "/page-2"', href='/page-2'),

    # content will be rendered in this element
    html.Div(id='page-content')
])


@callback(Output('page-content', 'children'),
              [Input('url', 'pathname')])
def display_page(pathname):
    return html.Div([
        html.H3(f'You are on page {pathname}')
    ])


if __name__ == '__main__':
    app.run_server(debug=True)

In this example, the callback `display_page` receives the current pathname (the last part of the URL) of the page. The callback simply displays the `pathname` on page but it could use the `pathname` to display different content.

The `Link` element updates the `pathname` of the browser without refreshing the page. If you used a `html.A` element instead, the `pathname` would update but the page would refresh.

Here is what this example running looks like. Note how clicking on the `Link` doesn't refresh the page even though it updates the URL!

この例では、コールバック `display_page` がページの現在のパス名(URLの最後の部分)を受け取ります。コールバックは単純に `pathname` をページに表示しますが、`pathname` を使って別のコンテンツを表示することもできます。

`Link` 要素は、ページをリフレッシュすることなくブラウザの `pathname` を更新します。代わりに `html.A` 要素を使用した場合、`pathname` は更新されますが、ページは更新されます。

この実行例は次のようなものです。`Link` をクリックしても、URLは更新されるのにページが更新されないことに注意してください!

### Example With Different Pages

You can modify the previous example to display different pages depending on the URL:

### ページが異なる場合の例

先ほどの例を修正して、URLによって異なるページを表示することができます。

In [ ]:
from dash import Dash, dcc, html, Input, Output, callback

app = Dash(__name__, suppress_callback_exceptions=True)

app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    html.Div(id='page-content')
])


index_page = html.Div([
    dcc.Link('Go to Page 1', href='/page-1'),
    html.Br(),
    dcc.Link('Go to Page 2', href='/page-2'),
])

page_1_layout = html.Div([
    html.H1('Page 1'),
    dcc.Dropdown(['LA', 'NYC', 'MTL'], 'LA', id='page-1-dropdown'),
    html.Div(id='page-1-content'),
    html.Br(),
    dcc.Link('Go to Page 2', href='/page-2'),
    html.Br(),
    dcc.Link('Go back to home', href='/'),
])

@callback(Output('page-1-content', 'children'),
              [Input('page-1-dropdown', 'value')])
def page_1_dropdown(value):
    return f'You have selected {value}'


page_2_layout = html.Div([
    html.H1('Page 2'),
    dcc.RadioItems(['Orange', 'Blue', 'Red'], 'Orange', id='page-2-radios'),
    html.Div(id='page-2-content'),
    html.Br(),
    dcc.Link('Go to Page 1', href='/page-1'),
    html.Br(),
    dcc.Link('Go back to home', href='/')
])

@callback(Output('page-2-content', 'children'),
              [Input('page-2-radios', 'value')])
def page_2_radios(value):
    return f'You have selected {value}'


# Update the index
@callback(Output('page-content', 'children'),
              [Input('url', 'pathname')])
def display_page(pathname):
    if pathname == '/page-1':
        return page_1_layout
    elif pathname == '/page-2':
        return page_2_layout
    else:
        return index_page
    # You could also return a 404 "URL not found" page here

if __name__ == '__main__':
    app.run_server(debug=True)

In this example, we're displaying different layouts through the `display_page` function. A few notes:

- Each page can have interactive elements even though those elements may not be in the initial view. Dash handles these "dynamically generated" components gracefully: as they are rendered, they will trigger the callbacks with their initial values.

- Since we're adding callbacks to elements that don't exist in the app.layout, Dash will raise an exception to warn us that we might be doing something wrong. In this case, we're adding the elements through a callback, so we can ignore the exception by setting `suppress_callback_exceptions=True`. It is also possible to do this without suppressing callback exceptions. See the example below for details.

- You can modify this example to import the different page's `layout`s in different files.

- This Dash Userguide that you're looking at is itself a multi-page Dash app, using these same principles.

この例では、`display_page`関数で異なるレイアウトを表示させています。いくつかの注意点があります。

- 各ページには、インタラクティブな要素がありますが、それらの要素は初期ビューにない場合があります。Dash はこれらの「動的に生成された」コンポーネントを優雅に扱います。それらがレンダリングされると、初期値でコールバックが起動します。

- app.layout に存在しない要素にコールバックを追加しているので、Dash は何か間違ったことをしているかもしれないと警告するために例外を発生させます。この場合、コールバックを通して要素を追加しているので、`suppress_callback_exceptions=True`を設定することで例外を無視することができます。また、コールバックの例外を抑制せずに行うことも可能です。詳しくは、以下の例を参照してください。

- この例を修正して、異なるページの `layout` を異なるファイルにインポートすることができます。

- この「Dashユーザーズガイド」は、これと同じ原理で、複数ページのDashアプリになっています。

### Dynamically Create a Layout for Multi-Page App Validation

Dash applies validation to your callbacks, which performs checks such as validating the types of callback arguments and checking to see whether the specified Input and Output components actually have the specified properties.

For full validation, all components within your callback must therefore appear in the initial layout of your app, and you will see an error if they do not. However, in the case of more complex Dash apps that involve dynamic modification of the layout (such as multi-page apps), not every component appearing in your callbacks will be included in the initial layout.

New in Dash 1.12 You can set `app.validation_layout` to a "complete" layout that contains all the components you'll use in any of the pages / sections. `app.validation_layout` must be a Dash component, not a function. Then set `app.layout` to just the index layout. In previous Dash versions there was a trick you could use to achieve the same result, checking `flask.has_request_context` inside a layout function - that will still work but is no longer recommended.

### 複数ページのアプリ検証のためのレイアウトを動的に作成する

Dashはコールバックにバリデーションを適用し、コールバック引数のタイプの検証や、指定されたInputコンポーネントやOutputコンポーネントが実際に指定されたプロパティを持つかどうかのチェックなどを実行します。

したがって、完全な検証を行うには、コールバック内のすべてのコンポーネントがアプリの初期レイアウトに表示される必要があり、表示されない場合はエラーが表示されます。ただし、レイアウトを動的に変更するような複雑なDashアプリ（複数ページのアプリなど）の場合、コールバック内のすべてのコンポーネントが初期レイアウトに含まれるわけではありません。

Dash 1.12 の新機能 `app.validation_layout` を、ページ/セクションで使用するすべてのコンポーネントを含む "完全な" レイアウトに設定することができます。`app.validation_layout` は関数ではなく、Dashのコンポーネントである必要があります。それから、`app.layout`をインデックスレイアウトだけに設定します。以前のバージョンのDashでは、レイアウト関数の中で `flask.has_request_context` をチェックすることで、同じ結果を得ることができましたが、これはまだ動作しますが、もう推奨されません。

In [ ]:
from dash import Dash, html, dcc, Input, Output, State, callback

app = Dash(__name__)

url_bar_and_content_div = html.Div([
    dcc.Location(id='url', refresh=False),
    html.Div(id='page-content')
])

layout_index = html.Div([
    dcc.Link('Navigate to "/page-1"', href='/page-1'),
    html.Br(),
    dcc.Link('Navigate to "/page-2"', href='/page-2'),
])

layout_page_1 = html.Div([
    html.H2('Page 1'),
    dcc.Input(id='input-1-state', type='text', value='Montreal'),
    dcc.Input(id='input-2-state', type='text', value='Canada'),
    html.Button(id='submit-button', n_clicks=0, children='Submit'),
    html.Div(id='output-state'),
    html.Br(),
    dcc.Link('Navigate to "/"', href='/'),
    html.Br(),
    dcc.Link('Navigate to "/page-2"', href='/page-2'),
])

layout_page_2 = html.Div([
    html.H2('Page 2'),
    dcc.Dropdown(['LA', 'NYC', 'MTL'], 'LA', id='page-2-dropdown'),
    html.Div(id='page-2-display-value'),
    html.Br(),
    dcc.Link('Navigate to "/"', href='/'),
    html.Br(),
    dcc.Link('Navigate to "/page-1"', href='/page-1'),
])

# index layout
app.layout = url_bar_and_content_div

# "complete" layout
app.validation_layout = html.Div([
    url_bar_and_content_div,
    layout_index,
    layout_page_1,
    layout_page_2,
])


# Index callbacks
@callback(Output('page-content', 'children'),
              Input('url', 'pathname'))
def display_page(pathname):
    if pathname == "/page-1":
        return layout_page_1
    elif pathname == "/page-2":
        return layout_page_2
    else:
        return layout_index


# Page 1 callbacks
@callback(Output('output-state', 'children'),
              Input('submit-button', 'n_clicks'),
              State('input-1-state', 'value'),
              State('input-2-state', 'value'))
def update_output(n_clicks, input1, input2):
    return f'The Button has been pressed {n_clicks} times. \
            Input 1 is {input1} and Input 2 is {input2}'


# Page 2 callbacks
@callback(Output('page-2-display-value', 'children'),
              Input('page-2-dropdown', 'value'))
def display_value(value):
    return f'You have selected {value}'


if __name__ == '__main__':
    app.run_server(debug=True)

### Structuring a Multi-Page App

Earlier examples show each multi-page app created within a single Python file. For bigger apps, a structure with multiple files may make the app easier to manage.

### 複数ページのアプリを構成する

先の例では、複数ページのアプリをそれぞれ1つのPythonファイル内に作成しました。大きなアプリの場合、複数のファイルを持つ構造の方がアプリの管理がしやすいかもしれません。

#### One Page Per File

One way to structure a multi-page app is to have each page as a separate app imported in the main app (`app.py`). In the following example, we build our app with two pages `pages/page1.py` and `pages/page2.py`. More pages (for example, `pages/page3.py`) can easily be added to this structure.

#### 1ファイル1ページ

複数ページのアプリを構成する1つの方法は、各ページをメインアプリ(`app.py`)にインポートされた別のアプリとして持つことです。次の例では、2つのページ `pages/page1.py` と `pages/page2.py` でアプリを構築しています。さらに多くのページ(例えば `pages/page3.py`) を、この構造に簡単に追加することができます。

```bash
- app.py
- pages
   |-- __init__.py
   |-- page1.py
   |-- page2.py
```

`pages/page1.py`

In [ ]:
from dash import dcc, html, Input, Output, callback

layout = html.Div([
    html.H3('Page 1'),
    dcc.Dropdown(
        {f'Page 1 - {i}': f'{i}' for i in ['New York City', 'Montreal', 'Los Angeles']},
        id='page-1-dropdown'
    ),
    html.Div(id='page-1-display-value'),
    dcc.Link('Go to Page 2', href='/page2')
])


@callback(
    Output('page-1-display-value', 'children'),
    Input('page-1-dropdown', 'value'))
def display_value(value):
    return f'You have selected {value}'

`pages/page2.py`

In [ ]:
from dash import dcc, html, Input, Output, callback

layout = html.Div([
    html.H3('Page 2'),
    dcc.Dropdown(
        {f'Page 2 - {i}': f'{i}' for i in ['London', 'Berlin', 'Paris']},
        id='page-2-dropdown'
    ),
    html.Div(id='page-2-display-value'),
    dcc.Link('Go to Page 1', href='/page1')
])


@callback(
    Output('page-2-display-value', 'children'),
    Input('page-2-dropdown', 'value'))
def display_value(value):
    return f'You have selected {value}'

`app.py`

In `app.py` we import `page1` and `page2`. When you run `app.py` it loads the layout from `page1.py` if you go to the pathname `/page1` and the layout from `page2.py` if you go to `/page2`.

`app.py` では、 `page1` と `page2` をインポートしています。`app.py` を実行すると、パス名 `/page1` にアクセスした場合は `page1.py` から、パス名 `/page2` にアクセスした場合は `page2.py` からレイアウトがロードされます。

In [ ]:
from dash import Dash, dcc, html, Input, Output, callback
from pages import page1, page2


app = Dash(__name__, suppress_callback_exceptions=True)
server = app.server

app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    html.Div(id='page-content')
])


@callback(Output('page-content', 'children'),
              Input('url', 'pathname'))
def display_page(pathname):
    if pathname == '/page1':
        return page1.layout
    elif pathname == '/page2':
        return page2.layout
    else:
        return '404'

if __name__ == '__main__':
    app.run_server(debug=True)

#### Flat Project Structure

Another option for a multi-page structure is a flat project layout with callbacks and layouts in separate files:

##### File structure:

#### フラットなプロジェクト構造

また、複数ページ構成の場合、コールバックとレイアウトを別ファイルにしたフラットなプロジェクトレイアウトという選択肢もあります。

##### ファイル構成:

```bash
- app.py
- callbacks.py
- layouts.py
```

`app.py`

In [ ]:
from dash import Dash, dcc, html, Input, Output, callback

from layouts import layout1, layout2
import callbacks

app = Dash(__name__, suppress_callback_exceptions=True)
server = app.server

app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    html.Div(id='page-content')
])

@callback(Output('page-content', 'children'),
              Input('url', 'pathname'))
def display_page(pathname):
    if pathname == '/page1':
         return layout1
    elif pathname == '/page2':
         return layout2
    else:
        return '404'

if __name__ == '__main__':
    app.run_server(debug=True)

`callbacks.py`

In [ ]:
from dash import Input, Output, callback

@callback(
    Output('page-1-display-value', 'children'),
    Input('page-1-dropdown', 'value'))
def display_value(value):
    return f'You have selected {value}'

@callback(
    Output('page-2-display-value', 'children'),
    Input('page-2-dropdown', 'value'))
def display_value(value):
    return f'You have selected {value}'

`layouts.py`

In [ ]:
from dash import dcc, html

layout1 = html.Div([
    html.H3('Page 1'),
    dcc.Dropdown(
        {f'Page 1 - {i}': f'{i}' for i in ['New York City', 'Montreal', 'Los Angeles']},
        id='page-1-dropdown'
    ),
    html.Div(id='page-1-display-value'),
    dcc.Link('Go to Page 2', href='/page2')
])

layout2 = html.Div([
    html.H3('Page 2'),
    dcc.Dropdown(
        {f'Page 2 - {i}': f'{i}' for i in ['London', 'Berlin', 'Paris']},
        id='page-2-dropdown'
    ),
    html.Div(id='page-2-display-value'),
    dcc.Link('Go to Page 1', href='/page1')
])

>The multi-page app examples here use the decorator `@callback`, which is new with Dash 2.0. It simplifies creating multi-page apps. More changes to make it even easier to create multi-page apps are also on the way in future versions of Dash. Big thanks to the Dash Community for contributing to these features with Dash Labs. [Read more about other ways we're working on simplifying multi-page apps](https://community.plotly.com/t/dash-labs-1-0-0-dash-pages-an-easier-way-to-make-multi-page-apps/58249/2?_gl=1*18jiu4l*_ga*MTU4NzAyNjYyMC4xNjY4NjM2MzMw*_ga_6G7EE0JNSC*MTY2OTU5Mzk2Ny41OC4xLjE2Njk1OTQ5MjMuMC4wLjA.).

>マルチページアプリケーションの例では、Dash 2.0 で新しく追加された `@callback` というデコレータを使用しています。これは Dash 2.0 の新機能で、複数ページのアプリを簡単に作成できるようにするものです。Dash の将来のバージョンでは、マルチページアプリをより簡単に作成できるようにするためのさらなる変更が予定されています。Dash コミュニティが Dash Labs でこれらの機能に貢献してくれていることに感謝します。[複数ページのアプリを簡単にするための他の方法については、こちらをご覧ください。](https://community.plotly.com/t/dash-labs-1-0-0-dash-pages-an-easier-way-to-make-multi-page-apps/58249/2?_gl=1*18jiu4l*_ga*MTU4NzAyNjYyMC4xNjY4NjM2MzMw*_ga_6G7EE0JNSC*MTY2OTU5Mzk2Ny41OC4xLjE2Njk1OTQ5MjMuMC4wLjA.)
